In [1]:
import time
from api.interface import *
from datetime import datetime

Let's start with Kafka

To run server & consumer, open three terminals in order (in kafka directory)
- bin/zookeeper-server-start.sh config/zookeeper.properties 
- bin/kafka-server-start.sh config/server.properties
- bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testing

In [2]:
# Start everything up as before
config = Config(
    provider = 'kafka', server = 'localhost:9092'
)
consumer = Consumer(config)
consumer.add_topic("fashion_mnist")

Run this to send a stream of images and indices to fashion_mnist:

```python pt3/simulated_client.py```

In [3]:
# Now, we can start streaming messages into a function that classifies the image:
# For now, we'll do it in the main thread so we can see the results live
# But this is blocking. In the next cell I'll trigger this in the OOP way so that it starts a thread
# automatically

import numpy as np
import pickle, codecs, json, torch

from cnn.model import generate_model_by_name
from cnn.api import predict_numpy_images
from api.utils import generate_consumer, direct_messages


model = generate_model_by_name(num_classes=10, input_size=[1, 28, 28], name="default")
# Load weights here

def classification_callback_function(message):
    index, image = json.loads(message.value.decode())
    image = np.array(image)
    output = predict_numpy_images([image], model)
    pred = torch.argmax(output)
    print(f"Image with id {index} classified as: {pred}")

    
con = generate_consumer(config, "fashion_mnist")
direct_messages(config, con, classification_callback_function)

# In fact starting this in a blocking way gives this error...

Image with id 6997 classified as: 7


AttributeError: '_MainThread' object has no attribute 'stopped'

Instead, what we should do is redirect our predictions to another kafka topic

Consumer.start_stream() does what we did above in an async way

Start a listening terminal for the ```fashion_predictions``` topic to see the results pour in!

In [9]:
import threading

results_producer = Producer(config)
results_producer.add_topic('fashion_predictions')

def classify_and_report_callback_function(msg):
    index, image = json.loads(msg.value.decode())
    img = np.array(image)
    output = predict_numpy_images([img], model)
    pred = torch.argmax(output).item()

    results_producer.send_message(
        message=(index, pred),
        topic_name="fashion_predictions"
    )

try:
    consumer.kill_stream("fashion_mnist", "classification")
except KeyError:
    pass

consumer.start_stream(
    topic_name="fashion_mnist", 
    callback_function=classify_and_report_callback_function, 
    stream_name="classification"
)

Already a topic named fashion_predictions!
Started streaming messages from fashion_mnist to <function classify_and_report_callback_function at 0x7f9a112f7160>
<StoppableThread(Thread-5, started 13092790272)> [[[-0.75960265  1.27379338 -1.48741386 -0.56738932  0.81829085
    0.53457755  1.23876495 -0.17928005  0.14657656  0.72388034
   -2.71968289 -1.76485195 -0.53831497  0.17135481 -1.33047533
   -0.79259134 -0.39360667  1.25164419  1.01697647  0.58998758
    1.26591676  1.48812716 -1.38919251  0.83547349  0.49162131
    0.8522375  -1.13137669 -0.29567789]
  [ 0.06724074  1.47404352 -1.82270803  1.16984328 -0.60803693
   -1.53563016 -0.5190653   1.25271441 -1.60297158  0.40783974
    1.32918267 -1.65076649  0.26732819 -0.3271635  -0.23803499
   -0.62058745 -0.87122179  0.59692082  0.42273299 -0.25323902
   -2.72189819  0.38471692  1.65056987  0.53607692 -1.2579445
    0.16539704 -2.2864332   1.21223532]
  [-0.12938853 -0.6966296   0.98278743 -0.01532684  1.17465184
   -1.13698677 -0.21